In [1]:
class Problem:
    '''
    Abstract base class for problem formulation that supports a heuristic function.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None
    
    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError
    
    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError
    
    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError
    
    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

    def heuristic(self, state):
        '''Returns the heuristic value of the given state, i.e., the estimated number of steps to the nearest goal state.'''
        raise NotImplementedError

In [2]:
class Node:
    '''Node data structure for search space bookkeeping.'''
    
    def __init__(self, state, parent, action, path_cost, heuristic):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.g = path_cost
        self.h = heuristic
        self.f = path_cost + heuristic

    @classmethod
    def root(cls, problem):
        '''Factory method to create the root node.'''
        init_state = problem.init_state
        return cls(init_state, None, None, 0, problem.heuristic(init_state))

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        child_state = problem.result(parent.state, action)
        return cls(
            child_state,
            parent,
            action,
            parent.g + problem.step_cost(parent.state, action),
            problem.heuristic(child_state))

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.g
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [3]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for _, _, node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

In [4]:
class SlidingPuzzle3x3(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}
    
    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves
    
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        expected_cost = 0
        for x, y in zip(state, self._goal_state):
            if x != y: expected_cost += 1
        return expected_cost

def _sliding_puzzle_3x3_visualizer(problem, state):
    '''Custom visualizer for the 3x3 sliding puzzle problem.'''
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

_visualizers[SlidingPuzzle3x3] = _sliding_puzzle_3x3_visualizer

In [5]:
problem = SlidingPuzzle3x3('12345678 ', '123 56478')

In [6]:
from heapq import heappush, heappop
from itertools import count

counter = count()

#Uniform-cost-search-algorithm
def UCS(problem, verbose=False):
    
    frontier = [(None, None, Node.root(problem))]
    explored = set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        _, _, node = heappop(frontier)
        if node.state in explored: continue
        if problem.goal_test(node.state):
            return solution(node)
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                heappush(frontier, (child.g, next(counter), child))